## scrape links for contest

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
from datetime import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
PATH = "C:\Program Files (x86)\chromedriver.exe" 

In [3]:
def get_tags(page_number): 

    tags = []

    if page_number == 1:
        url = "https://99designs.hk/logo-design/contests?sort=start-date%3Adesc&status=won&entry-level=0&mid-level=0&top-level=0&dir=desc&order=start-date"
    else:
        url = "https://99designs.hk/logo-design/contests?sort=start-date%3Adesc&status=won&entry-level=0&mid-level=0&top-level=0&page=" + str(page_number) + "&dir=desc&order=start-date"

    driver = webdriver.Chrome(PATH)
    driver.get(url)

    for i in range(30):
        index = i + 1 
        xml_path = '/html/body/div[1]/div[1]/div[4]/div/div/div[3]/div/div/div[1]/div['+str(index)+']/div/div[2]/div/div[1]/div[2]'
        tag = driver.find_element(By.XPATH, xml_path).text
        # print(tags)
        tags.append(tag)

    driver.close()
    return tags

In [4]:
def make_soup(page_number):

    kv = {'user-agent': 'Mozilla/5.0'}
    if page_number == 1:
        url = "https://99designs.hk/logo-design/contests?sort=start-date%3Adesc&status=won&entry-level=0&mid-level=0&top-level=0&dir=desc&order=start-date"
    else:
        url = "https://99designs.hk/logo-design/contests?sort=start-date%3Adesc&status=won&entry-level=0&mid-level=0&top-level=0&page=" + str(page_number) + "&dir=desc&order=start-date"
    r = requests.get(url, headers=kv, timeout=30)
    r.raise_for_status()
    r.status_code
    r.encoding = r.apparent_encoding
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup

In [5]:
def get_urls(page_number):
    
    href = []
    tags = []
    for i in tqdm(range(page_number)):
        # get url
        soup = make_soup(i)
        results = soup.findAll(name='a', attrs='listing-details__title__link')
        for result in results:
            href.append('https://99designs.hk'+result['href'])

    
        # get tags
        tag_one_page = get_tags(i)
        tags += tag_one_page
        

    
    df_href = pd.DataFrame({'href': href, 'tags': tags})
    df_href.to_csv('href_' + str(datetime.now().date()) + '.csv', index=False)

In [6]:
get_urls(20)


100%|██████████| 20/20 [21:36<00:00, 64.84s/it]
